Decision Transformer: Reinforcement Learning via Sequence Modeling
Recreated by : Austin Runkle, 

In this project we will be implementing the decision transformer

IMPORTS

Decision Transformer - For Continuous Action

In [ ]:
# R, s, a, t: returns -to -go , states , actions , or timesteps
# K: context length ( length of each input to DecisionTransformer )
# transformer : transformer with causal masking (GPT)
# embed_s , embed_a , embed_R : linear embedding layers
# embed_t : learned episode positional embedding
# pred_a : linear action prediction layer
# main model
def DecisionTransformer (R , s , a , t ):
    # compute embeddings for tokens
    pos_embedding = embed_t ( t ) # per - timestep ( note : not per - token )
    s_embedding = embed_s ( s ) + pos_embedding
    a_embedding = embed_a ( a ) + pos_embedding
    R_embedding = embed_R ( R ) + pos_embedding
    # interleave tokens as (R_1 , s_1 , a_1 , ... , R_K , s_K )
    input_embeds = stack ( R_embedding , s_embedding , a_embedding )
    # use transformer to get hidden states
    hidden_states = transformer ( input_embeds = input_embeds )
    # select hidden states for action prediction tokens
    a_hidden = unstack ( hidden_states ). actions
    # predict action
    return pred_a ( a_hidden )
# training loop
for (R , s , a , t ) in dataloader : # dims : ( batch_size , K, dim )
    a_preds = DecisionTransformer (R , s , a , t )
    loss = mean (( a_preds - a )**2) # L2 loss for continuous actions
    optimizer . zero_grad (); loss . backward (); optimizer . step ()
# evaluation loop
target_return = 1 # for instance , expert - level return
R , s , a , t , done = [ target_return ] , [ env . reset ()] , [] , [1] , False
while not done : # autoregressive generation / sampling
    # sample next action
    action = DecisionTransformer (R , s , a , t )[ -1] # for cts actions
    new_s , r , done , _ = env . step ( action )
    # append new tokens to sequence
    R = R + [ R [ -1] - r] # decrement returns -to -go with reward
    s , a , t = s + [ new_s ] , a + [ action ] , t + [ len ( R )]
    R , s , a , t = R [ - K :] , ... # only keep context length of K
